<a href="https://colab.research.google.com/github/ajw1587/Pytorch_Study/blob/main/16_CNN_MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install torch
!pip install torchvision

In [49]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [50]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [51]:
# MNIST dataset

mnist_train = dsets.MNIST(root = '../data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = '../data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [52]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = 100,
                                          shuffle = True,
                                          drop_last = True)

In [53]:
class MNIST_model(nn.Module):
  def __init__(self):
    super(MNIST_model, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    
    self.fc1 = nn.Linear(3 * 3 * 128, 625, bias = True)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [54]:
model = MNIST_model().to(device)
model

MNIST_model(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [55]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [56]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    x = X.to(device)
    y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(x)

    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / totla_batch

  print('[Epoch: {}] cost = {}'.format(epoch, avg_cost))
print('Learning Finished')

[Epoch: 0] cost = 0.15955929458141327
[Epoch: 1] cost = 0.044457174837589264
[Epoch: 2] cost = 0.030418358743190765
[Epoch: 3] cost = 0.022999292239546776
[Epoch: 4] cost = 0.01726643182337284
[Epoch: 5] cost = 0.014983483590185642
[Epoch: 6] cost = 0.012541763484477997
[Epoch: 7] cost = 0.011052276007831097
[Epoch: 8] cost = 0.011060615070164204
[Epoch: 9] cost = 0.007596713025122881
[Epoch: 10] cost = 0.007356793154031038
[Epoch: 11] cost = 0.006953639443963766
[Epoch: 12] cost = 0.005230082664638758
[Epoch: 13] cost = 0.007439941633492708
[Epoch: 14] cost = 0.004879760090261698
Learning Finished


In [57]:
with torch.no_grad():
  x_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.targets.to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: {}'.format(accuracy.item()))

Accuracy: 0.9838999509811401
